<a href="https://colab.research.google.com/github/ParitoshP702/Bilevel-Optimization/blob/main/MNIST_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import scipy as spy

from operator import itemgetter

from tqdm import tqdm

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

from keras import backend as K

In [ ]:
(X_train,Y_train) , (X_test,Y_test) = tf.keras.datasets.mnist.load_data(path = "mnist.npz")

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
train_count = 5000
eval_count = 2500

In [ ]:
X_test = X_test[:eval_count,:,:]

In [ ]:
x_test = X_test.reshape(X_test.shape[0],-1)

In [ ]:
y_test = Y_test[:eval_count]

In [ ]:
Y_train.shape

(60000,)

In [ ]:
x_train = X_train[:train_count,:,:]
x_eval = X_train[train_count:train_count+eval_count,:,:]
y_train = Y_train[:train_count]
y_eval = Y_train[train_count:train_count+eval_count]

In [ ]:
x_train = x_train.reshape(x_train.shape[0],-1)
x_eval = x_eval.reshape(x_eval.shape[0],-1)

In [ ]:
x_train.shape

(5000, 784)

In [ ]:
y_training = np.zeros(shape = (len(y_train),10))

In [ ]:
for i in range(len(y_train)):
  # y_training[i][y_train_array[i]] = 1
  for j in range(10):
    if j != (y_train[i]):
      y_training[i][j] = 0.0
    else:
      y_training[i][j] = 1.0

In [ ]:
y_val_training = np.empty(shape = (len(y_eval),10),dtype = float)

In [ ]:
for i in range(len(y_eval)):
   for j in range(10):
    if j != (y_eval[i]):
      y_val_training[i][j] = 0.0
    else:
      y_val_training[i][j] = 1.0

In [ ]:
number_of_layers = 5
neuron_per_layer = 50
activation_function = "sigmoid"
optimizer = "Adam"

In [ ]:
model = Sequential()
model.add(Dense(units=3, input_dim=784))

for _ in range(number_of_layers):
  model.add(Dense(units=neuron_per_layer, activation=activation_function, kernel_regularizer = tf.keras.regularizers.L2(1e-10)))
model.add(Dense(units = 10,  activation = 'softmax', kernel_regularizer = tf.keras.regularizers.L2(1e-10)))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizer, metrics=['accuracy'])
model.fit(x_train, y_training, batch_size = 64, epochs = 10)

loss_score, accuracy_score = model.evaluate(x_eval,y_val_training, verbose = 0)

Epoch 1/10
79/79 [==============================] - 1s 3ms/step - loss: 2.3106 - accuracy: 0.1058
Epoch 2/10
79/79 [==============================] - 0s 3ms/step - loss: 2.2806 - accuracy: 0.1472
Epoch 3/10
79/79 [==============================] - 0s 3ms/step - loss: 2.0031 - accuracy: 0.3062
Epoch 4/10
79/79 [==============================] - 0s 3ms/step - loss: 1.5946 - accuracy: 0.3462
Epoch 5/10
79/79 [==============================] - 0s 3ms/step - loss: 1.4137 - accuracy: 0.3906
Epoch 6/10
79/79 [==============================] - 0s 3ms/step - loss: 1.3338 - accuracy: 0.4080
Epoch 7/10
79/79 [==============================] - 0s 3ms/step - loss: 1.2826 - accuracy: 0.4430
Epoch 8/10
79/79 [==============================] - 0s 3ms/step - loss: 1.2418 - accuracy: 0.4810
Epoch 9/10
79/79 [==============================] - 0s 3ms/step - loss: 1.1956 - accuracy: 0.5234
Epoch 10/10
79/79 [==============================] - 0s 3ms/step - loss: 1.1438 - accuracy: 0.5734


In [ ]:
loss_score

1.1546335220336914

In [ ]:
# y_p_training = model(x_train,training = True)

In [ ]:
# loss_object(y_p_training,y_training)

In [ ]:
y_training

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
skip_length = len(model.layers[0].weights[0].numpy().reshape(-1)) + len(model.layers[0].weights[1].numpy().reshape(-1))

In [ ]:
final_hessian_list = []

In [ ]:
with tf.GradientTape(persistent = True) as tape1:
  with tf.GradientTape(persistent = True) as tape2:
    loss_object = tf.keras.losses.CategoricalCrossentropy()
    y_pred_array = model(x_train,training = True)
    loss = loss_object(y_training,y_pred_array)
  g = tape2.gradient(loss, model.trainable_variables)
for i in range(len(g)):
  # reshaped_grad = tf.reshape(g[i], [-1])
  h = tape1.jacobian(g[i],model.trainable_variables)
  final_hessian_list.append(h)

In [ ]:
def compute_gradient(x_target,y_target,model):##general function which returns the list of gradient vector as an numpy array
  with tf.GradientTape() as tape:
    loss_object = tf.keras.losses.MeanSquaredError()
    y_pred_array = model(x_target,training = True)
    loss = loss_object(y_target,y_pred_array)
  g = tape.gradient(loss,model.trainable_variables)
  final_grad_list = []
  for i in range(len(g)):
    grad_array = tf.make_ndarray(tf.make_tensor_proto(g[i]))
    if i%2==0:
      grad_shape = grad_array.shape
      for j in range(grad_shape[0]):
        for k in range(grad_shape[1]):
          final_grad_list.append(grad_array[j][k])
    else:
      length = grad_array.shape[0]
      for j in range(length):
        final_grad_list.append(grad_array[j])
  return np.array(final_grad_list)


In [ ]:
grad_validation = compute_gradient(x_eval,y_val_training,model)

In [ ]:
#reshaping the double dimensional list of tensors into a matrix
hessian_matrix = np.empty(shape = (1,1),dtype = float)
for i in range(len(final_hessian_list)):
  hess_col_mat = np.empty(shape = (1,1),dtype = float)
  for j in range(len(final_hessian_list[i])):
    hess_array = tf.make_ndarray(tf.make_tensor_proto(final_hessian_list[i][j]))
    hess_shape = hess_array.shape
    if i%2 == 0:
       if j%2 == 0:
         hess_array = hess_array.reshape(hess_shape[0]*hess_shape[1],hess_shape[2]*hess_shape[3])
       else:
         hess_array = hess_array.reshape(hess_shape[0]*hess_shape[1],hess_shape[2])
    else:
      if j%2 == 0:
        hess_array = hess_array.reshape(hess_shape[0],hess_shape[1]*hess_shape[2])
      else:
        hess_array = hess_array
    if j==0 :
      hess_col_mat = hess_array
    else:
      hess_col_mat = np.concatenate((hess_col_mat,hess_array),axis = 1)
  if i==0:
    hessian_matrix = hess_col_mat
  else:
    hessian_matrix= np.concatenate((hessian_matrix,hess_col_mat),axis = 0)



In [ ]:
hessian_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.07595121,
        -0.00041618, -0.03217039],
       [ 0.        ,  0.        ,  0.        , ..., -0.00041618,
         0.06883408, -0.00046167],
       [ 0.        ,  0.        ,  0.        , ..., -0.03217039,
        -0.00046167,  0.07187487]], dtype=float32)

In [ ]:
weights_list = []
weights_list_new = []

In [ ]:
for i in range(len(model.weights)):
  weights_array = tf.make_ndarray(tf.make_tensor_proto(model.weights[i]))
  if i%2 == 0:
    shape_array = weights_array.shape
    for j in range(shape_array[0]):
      for k in range(shape_array[1]):
        weights_list.append(weights_array[j][k])
        if len(weights_list_new) < skip_length:
          weights_list_new.append(0)
        else:
          weights_list_new.append(weights_array[j][k])
  else:
    lgt = weights_array.shape[0]
    for j in range(lgt):
      weights_list.append(weights_array[j])
      if len(weights_list_new) < skip_length:
        weights_list_new.append(0)
      else:
        weights_list_new.append(weights_array[j])

In [ ]:
final_weights_array = np.array(weights_list)
final_weights_array_new = np.array(weights_list_new)

In [ ]:
l = len(final_weights_array)

In [ ]:
l

5315

In [ ]:
weight_array_vec = final_weights_array_new.reshape(l,1)/len(y_train)

In [ ]:
hessian_col_mat = np.concatenate((weight_array_vec,hessian_matrix),axis = 1)

In [ ]:
weight_array_withreg = np.concatenate(([[0]],final_weights_array_new.reshape(1,l)),axis = 1)/len(y_train)

In [ ]:
hessian_mat_with_reg = np.concatenate((weight_array_withreg,hessian_col_mat),axis = 0)

In [ ]:
hessian_mat_with_reg

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.03743894e-06, -2.72571202e-06, -1.80651457e-07],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-2.03743894e-06,  0.00000000e+00,  0.00000000e+00, ...,
         8.88276994e-02, -2.74284626e-03, -1.96078978e-02],
       [-2.72571202e-06,  0.00000000e+00,  0.00000000e+00, ...,
        -2.74284626e-03,  7.95235187e-02, -2.37949542e-03],
       [-1.80651457e-07,  0.00000000e+00,  0.00000000e+00, ...,
        -1.96078978e-02, -2.37949542e-03,  8.23247358e-02]])

In [ ]:
grad_validation_new=[]

In [ ]:
grad_validation_new = np.concatenate(([[0]],grad_validation.reshape(1,l)),axis = 1)

In [ ]:
##obtaining the new gradient array
grad_validation_new = grad_validation_new.reshape(-1)

In [ ]:
final_weights_array

array([ 0.07031602,  0.02224601,  0.01752656, ..., -0.01018719,
       -0.01362856, -0.00090326], dtype=float32)

In [ ]:
final_weights_array

array([ 0.07031602,  0.02224601,  0.01752656, ..., -0.01018719,
       -0.01362856, -0.00090326], dtype=float32)

In [ ]:
ub = [10 for i in range(l+1)]

In [ ]:
lb = []
for i in range(l+1):
  if i==0:
    lb.append(1e-5)
  else:
    lb.append(-10)

In [ ]:
pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gurobipy as gp

In [ ]:
np.linalg.matrix_rank(hessian_mat_with_reg)

4921

In [ ]:
from gurobipy import *

In [ ]:
params = {
"WLSACCESSID": '753e7886-7142-449d-8baa-d41ca78716ef',
"WLSSECRET": '880d2525-364b-41d0-ac23-6dcf7ad15312',
"LICENSEID": 914249,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
m = gp.Model(env=env)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 914249
Academic license - for non-commercial use only - registered to ppankaj21@iitk.ac.in


In [ ]:
# m = gp.Model()
x = m.addMVar((l+1,),lb = lb, ub = ub )
m.setObjective(grad_validation_new@x)
# m.addConstr(hessian_mat_with_reg@x == 0)
m.addConstr(hessian_mat_with_reg@x <= 0.1)
m.addConstr(hessian_mat_with_reg@x >= -0.1)
x.PStart = np.zeros(l+1)
# GRBModel.Set(Pstart = np.zeros(l+1))
m.optimize()

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning on update mode = 1 after new variables or constraints added:
Setting LP warm start basis or start ignored
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Academic license - for non-commercial use only - registered to ppankaj21@iitk.ac.in
Optimize a model with 10632 rows, 5316 columns and 47009426 nonzeros
Model fingerprint: 0xd443a9a7
Coefficient statistics:
  Matrix range     [1e-13, 9e-02]
  Objective range  [2e-11, 5e-04]
  Bounds range     [1e-05, 1e+01]
  RHS range        [1e-01, 1e-01]
Presolve removed 1400 rows and 377 columns (presolve time = 7s) ...
Presolve removed 1400 rows and 377 columns (presolve time = 14s) ...
Presolve removed 1400 rows and 377 columns (presolve time = 21s) ...
Presolve removed 6016 rows and 377 col

In [ ]:
x

<MVar (5316,)>
array([<gurobi.Var C0 (value 10.0)>, <gurobi.Var C1 (value -10.0)>,
       <gurobi.Var C2 (value -10.0)>, ...,
       <gurobi.Var C5313 (value -10.0)>, <gurobi.Var C5314 (value 10.0)>,
       <gurobi.Var C5315 (value -10.0)>])

In [ ]:
weight_array_with_reg = np.concatenate(([[1e-10]],final_weights_array.reshape(1,l)),axis = 1).reshape(-1)

In [ ]:
weight_array_with_reg

array([ 1.00000000e-10,  7.03160241e-02,  2.22460106e-02, ...,
       -1.01871947e-02, -1.36285601e-02, -9.03257285e-04])

In [ ]:
descent_factors = []
for i in range(-100,20000,200):
  descent_factors.append(i*1e-3)
descent_factors = np.array(descent_factors)

In [ ]:
weight_sample_space_matrix = np.empty(shape = (len(descent_factors),len(weight_array_with_reg)),dtype = float)

In [ ]:
x

<MVar (5316,)>
array([<gurobi.Var C0 (value 10.0)>, <gurobi.Var C1 (value -10.0)>,
       <gurobi.Var C2 (value -10.0)>, ...,
       <gurobi.Var C5313 (value -10.0)>, <gurobi.Var C5314 (value 10.0)>,
       <gurobi.Var C5315 (value -10.0)>])

In [ ]:
final_weights_array

array([ 0.07031602,  0.02224601,  0.01752656, ..., -0.01018719,
       -0.01362856, -0.00090326], dtype=float32)

In [ ]:
all_vars = m.getVars()
values = m.getAttr("x",all_vars)

In [ ]:
values = np.array(values)

In [ ]:
values = values/np.linalg.norm(values)

In [ ]:
values

array([ 0.01392511, -0.01392511, -0.01392511, ..., -0.01392511,
        0.01392511, -0.01392511])

In [ ]:
for i in range(len(descent_factors)):
  weight_sample_space_matrix[i] =weight_array_with_reg+ values*descent_factors[i]

In [ ]:
weight_sample_space_matrix

array([[-0.00139251,  0.07170853,  0.02363852, ..., -0.00879468,
        -0.01502107,  0.00048925],
       [ 0.00139251,  0.06892351,  0.0208535 , ..., -0.01157971,
        -0.01223605, -0.00229577],
       [ 0.00417753,  0.06613849,  0.01806848, ..., -0.01436473,
        -0.00945103, -0.00508079],
       ...,
       [ 0.27153958, -0.20122355, -0.24929357, ..., -0.28172677,
         0.25791102, -0.27244283],
       [ 0.2743246 , -0.20400857, -0.25207859, ..., -0.28451179,
         0.26069604, -0.27522786],
       [ 0.27710962, -0.2067936 , -0.25486361, ..., -0.28729681,
         0.26348106, -0.27801288]])

In [ ]:
def validation_loss_computation(weight_and_reg_array):##function which computes the loss score of the model corresponding to given weights

  model_demo = Sequential()
  model_demo.add(Dense(units = 3, input_dim = 784))
  for i in range(number_of_layers):
       model_demo.add(Dense(units = neuron_per_layer, activation = activation_function, kernel_regularizer = tf.keras.regularizers.L2(weight_and_reg_array[0])))
  model_demo.add(Dense(units = 10,activation = "softmax", kernel_regularizer = tf.keras.regularizers.L2(weight_and_reg_array[0])))
  model_demo.compile(loss = "mean_squared_error", optimizer = optimizer, metrics = ["accuracy"])
  weight_tracker = 1##as "weight_and_reg_array" is a one dimensional array it keeps track of the indices of the array
  for i in range(len(model_demo.layers)):##changing the weights of the model layer wise
    orignal_weight_list = model.layers[i].weights
    array_1 = orignal_weight_list[0].numpy()##array corresponding to the weight matrix
    array_2 = orignal_weight_list[1].numpy()##array corresponding to the bias vector
    array_1_new = weight_and_reg_array[weight_tracker:weight_tracker+array_1.shape[0]*array_1.shape[1]]
    weight_tracker += array_1.shape[0]*array_1.shape[1]##updating the weight tracker
    array_2_new = weight_and_reg_array[weight_tracker:weight_tracker + array_2.shape[0]]
    weight_tracker += array_2.shape[0] #updating the weight tracker
    array_1_new = array_1_new.reshape(array_1.shape) ##new weight matrix
    array_2_new = array_2_new.reshape(array_2.shape) ##new bias vector
    list_of_new_array = [] ##list of the new weight matrix and the new bias vector
    list_of_new_array.append(array_1_new)
    list_of_new_array.append(array_2_new)
    model_demo.layers[i].set_weights(list_of_new_array) ##appending the new weights into the given layer of the model
  y_pred_array = model_demo(np.array(x_eval),training = False)
  y_pred_training = model_demo(np.array(x_train),training = False)
  loss = loss_object(y_val_training,y_pred_array)
  loss_t = loss_object(y_training,y_pred_training)
  # loss1,_ = model_demo.evaluate(x_eval,y_eval,verbose= 0)
  # loss2,_= model_demo.evaluate(x_train,y_train,verbose = 0)
  return loss,loss_t,model_demo

In [ ]:
loss_array_valid = np.empty(shape = len(descent_factors),dtype = float)##array to contain the training losses
loss_array_train = np.empty(shape = len(descent_factors),dtype = float)##array to contain the validation losses

for i in range(len(loss_array_valid)):
  loss_array_valid[i] ,loss_array_train[i],_= validation_loss_computation(weight_sample_space_matrix[i])


In [ ]:
loss_array_valid.min()

1.7380784749984741

In [ ]:
loss_array_train.min()

1.73578941822052

In [ ]:
y_pred_array = model(x_eval,training = True)

In [ ]:
loss_object(y_val_training,y_pred_array)

<tf.Tensor: shape=(), dtype=float32, numpy=1.7885053>

In [ ]:
loss_array_valid.argmin()

24

In [ ]:
descent_factors[loss_array_valid.argmin()]

4.7

In [ ]:
descent_factors[loss_array_train.argmin()]

In [ ]:
pd.DataFrame([loss_array_valid,descent_factors,loss_array_train]).transpose().to_csv('temp(MNIST_new).csv')

In [ ]:
len(descent_factors)

In [ ]:
loss_array_train.argmin()

In [ ]:
ideal_weight_array = weight_sample_space_matrix[loss_array_valid.argmin()]

#Best weights result on the testing dataset

In [ ]:
_,_,best_model = validation_loss_computation(ideal_weight_array)

In [ ]:
y_pred_testing = best_model(x_test,training = True)

In [ ]:
y_testing = np.zeros(shape = (len(y_test),10))

In [ ]:
for i in range(len(y_test)):
   for j in range(10):
    if j != (y_test[i]):
      y_testing[i][j] = 0.0
    else:
      y_testing[i][j] = 1.0

In [ ]:
loss_object(y_testing,y_pred_testing)

In [ ]:
y_pred_test = model(x_test,training = True)

In [ ]:
loss_object(y_testing,y_pred_test)

In [ ]:
loss_object